In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
import xarray as xr
import xesmf
import numpy as np
import utils.geo as geo
import glob
from dask.diagnostics import ProgressBar

In [6]:
regrid = True

In [8]:
rootdir = '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0'
model = '*'
glob.glob('/'.join([rootdir,model,'*']))

['/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/COSMOS/GR30.nc',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/COSMOS/COSMOS-landveg_r2413',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/CESM/CESM1.2_CAM5',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/GFDL/GFDL_CM2.1',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/INMCM/INM-CM4-8',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/MIROC/MIROC4m',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/IPSL/IPSLCM5A2',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/NorESM1_F/deepmip_sens_4xCO2',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/NorESM1_F/piControl',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/NorESM1_F/deepmip_sens_2xCO2',
 '/Volumes

In [32]:
rootdir = '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0'
modelconfig = 'HadCM3/HadCM3BL_M2.1aN'
try:
    [model,config]=modelconfig.split('/')
except:
    model,config = modelconfig, modelconfig
experiment = '*'
version = 'v1.0'
variable = 'so'
averaging = 'mean'

In [34]:
filename_pre = '-'.join([config,experiment,variable,version])
filename = '.'.join([filename_pre,averaging,'nc'])
# file structure error correction
if model=='NorESM1_F':
    path = '/'.join([rootdir,model,experiment,version,filename])
else:
    path = '/'.join([rootdir,model,config,experiment,version,filename])
path

'/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/HadCM3/HadCM3BL_M2.1aN/*/v1.0/HadCM3BL_M2.1aN-*-so-v1.0.mean.nc'

In [36]:
paths = glob.glob(path)
# additional files present in COSMOS
paths = [path for path in paths if 'r122' not in path]
paths

['/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/HadCM3/HadCM3BL_M2.1aN/deepmip_stand_3xCO2/v1.0/HadCM3BL_M2.1aN-deepmip_stand_3xCO2-so-v1.0.mean.nc',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/HadCM3/HadCM3BL_M2.1aN/piControl/v1.0/HadCM3BL_M2.1aN-piControl-so-v1.0.mean.nc',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/HadCM3/HadCM3BL_M2.1aN/deepmip_sens_2xCO2/v1.0/HadCM3BL_M2.1aN-deepmip_sens_2xCO2-so-v1.0.mean.nc',
 '/Volumes/DeepMIP_Model_Output_read/DeepMIP-Eocene/User_Model_Database_v1.0/HadCM3/HadCM3BL_M2.1aN/deepmip_sens_1xCO2/v1.0/HadCM3BL_M2.1aN-deepmip_sens_1xCO2-so-v1.0.mean.nc']

In [38]:
if variable in ['tos','thetao','so']:
    dlon,dlat = 1,1
elif variable in ['tas','ta']:
    dlon,dlat = 3,2

grid = xr.Dataset()
lons = dlon/2 + np.arange(0,360,dlon)
lats = dlat/2 -90 + np.arange(0,180,dlat)
grid['lon'] = xr.DataArray(data=lons, dims=('x'), coords={'x':lons})
grid['lat'] = xr.DataArray(data=lats, dims=('y'), coords={'y':lats})
ds,xgrid = geo.get_xgcm_horizontal(grid,
                                   axes_dims_dict={'X':'x','Y':'y'},
                                   periodic=['X'],
                                   boundary_discontinuity={'x':360})

if variable in ['tos','thetao','so']:
    target_zs = np.array([5.000000e+00, 1.500000e+01, 2.500000e+01, 3.500000e+01, 4.500000e+01,
                           5.500000e+01, 6.500000e+01, 7.500000e+01, 8.500000e+01, 9.500000e+01,
                           1.050000e+02, 1.150000e+02, 1.250000e+02, 1.350000e+02, 1.450000e+02,
                           1.550000e+02, 1.650000e+02, 1.750000e+02, 1.850000e+02, 1.950000e+02,
                           2.050000e+02, 2.150000e+02, 2.250000e+02, 2.361228e+02, 2.506000e+02,
                           2.706208e+02, 2.983049e+02, 3.356756e+02, 3.846343e+02, 4.469366e+02,
                           5.241706e+02, 6.177363e+02, 7.288285e+02, 8.584215e+02, 1.007257e+03,
                           1.175835e+03, 1.364406e+03, 1.572971e+03, 1.801279e+03, 2.048829e+03,
                           2.314879e+03, 2.598456e+03, 2.898365e+03, 3.213206e+03, 3.541390e+03,
                           3.881162e+03, 4.230621e+03, 4.587743e+03, 4.950409e+03, 5.316429e+03])
    top = target_zs[-1]+np.diff(target_zs)[-1]
    bottom = 0
elif variable in ['tas','ta']:
    target_zs = np.sort(np.array([1000.,  950.,  900.,  850.,  800.,  750.,  700.,  650.,  600.,  550.,
                            500.,  450.,  400.,  350.,  300.,  250.,  200.,  150.,  100.,   70.,
                             50.,   30.,   20.,   10.]))
    top = 1050
    bottom = 5
mid_zs = np.append(bottom,0.5*(target_zs[:-1]+target_zs[1:]))
mid_zs = np.append(mid_zs,top)
dz = xr.DataArray(np.diff(mid_zs),dims='z',coords={'z':target_zs})

grid['area'] = ds['rC']
grid['dz'] = dz
grid['volume'] = grid['area']*grid['dz']

gridname = str(dlon)+'x'+str(dlat)
grid.to_netcdf('../data/processed/regridded/grid_'+gridname+'.nc')

latnames = ['TLAT','lat','geolat_t','latitude','nav_lat']
lonnames = ['TLONG','lon','geolon_t','longitude','nav_lon']
timenames = ['month','time','t','time_counter']
znames = ['z_t','st_ocean','lev','depth','depth_1','level']

/Users/gam24/anaconda3/envs/core/lib/python3.11/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg
/Users/gam24/anaconda3/envs/core/lib/python3.11/site-packages/xgcm/grid_ufunc.py:832: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  out_dim: grid._ds.dims[out_dim] for arg in out_core_dims for out_dim in arg


In [40]:
def preprocess(ds):
    path_elements = ds.encoding["source"].split('/')
    model = path_elements[5]
    print(model,end=', ')
    if model=='NorESM1_F': # muddled file structure for NorESM
        nconfig = 5
        nexperiment = 6
    else:
        nconfig = 6
        nexperiment = 7
    config = path_elements[nconfig]
    experiment = path_elements[nexperiment]
    print(config,end=', ')
    print(experiment)
    print(ds)
    
    # change variable names
    # lat
    lat = list(set(latnames).intersection(ds.coords))[0]
    if lat!='lat':
        ds = ds.rename({lat:'lat'})
    # lon
    lon = list(set(lonnames).intersection(ds.coords))[0]
    if lon!='lon':
        ds = ds.rename({lon:'lon'})
    # depth/pressure
    if len(list(set(znames).intersection(ds.coords)))>0:
        zdim = True
        z = list(set(znames).intersection(ds.coords))[0]
        if z!='z':
            ds = ds.rename({z:'z'})
    else:
        zdim=False
    # time
    time = list(set(timenames).intersection(ds.coords))[0]
    if time!='time':
        ds = ds.rename({time:'time'})
    if (model=='NorESM1_F') & (variable in ['thetao','tos']): # no seasonal info available for NorESM ocean
        timearray = np.arange(1)
    else:
        timearray = np.arange(12)
    ds = ds.assign_coords({'time':timearray})
    
    if regrid:
        ## horizontal regrid
        # initiate regridder
        regridder = xesmf.Regridder(ds,grid,'bilinear', periodic=True)
        # regrid dataset
        ds = regridder(ds)
        
        ## vertical regrid
        if zdim:
            if config=='CESM1.2_CAM5':
                ds['z']=ds['z']*1e-2 # conversion from cm
            ds = ds.interp({'z':target_zs})
    
    # place experiment as distinct dimension
    ds = ds.squeeze().expand_dims({'experiment':[str(experiment)]})
    # drop all other variables and rename main variable
    ds = ds[variable].to_dataset()
    
    print(ds)
    return ds

ds = xr.open_mfdataset(paths,preprocess=preprocess,decode_times=False)

HadCM3, HadCM3BL_M2.1aN, deepmip_stand_3xCO2
<xarray.Dataset> Size: 7MB
Dimensions:    (longitude: 96, latitude: 73, depth_1: 20, t: 12)
Coordinates:
  * longitude  (longitude) float32 384B 0.0 3.75 7.5 11.25 ... 348.8 352.5 356.2
  * latitude   (latitude) float32 292B -90.0 -87.5 -85.0 ... 85.0 87.5 90.0
  * depth_1    (depth_1) float32 80B 5.0 15.0 25.0 ... 4.577e+03 5.193e+03
  * t          (t) float32 48B 3.569e+04 3.572e+04 ... 3.599e+04 3.566e+04
Data variables:
    so         (t, depth_1, latitude, longitude) float32 7MB dask.array<chunksize=(12, 20, 73, 96), meta=np.ndarray>
Attributes:
    title:       HadCM3_model_output_for_DeepMIP_experiments_following_the_De...
    family:      HadCM3
    model:       HadCM3BL_M2.1aN
    experiment:  deepmip_stand_3xCO2
    version:     v1.0
<xarray.Dataset> Size: 156MB
Dimensions:     (experiment: 1, time: 12, x: 360, y: 180, z: 50)
Coordinates:
  * experiment  (experiment) object 8B 'deepmip_stand_3xCO2'
  * time        (time) int64 96B 

In [41]:
ds

<xarray.Dataset> Size: 622MB
Dimensions:     (experiment: 4, time: 12, z: 50, y: 180, x: 360)
Coordinates:
  * experiment  (experiment) object 32B 'deepmip_sens_1xCO2' ... 'piControl'
  * time        (time) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * x           (x) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * y           (y) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * z           (z) float64 400B 5.0 15.0 25.0 ... 4.588e+03 4.95e+03 5.316e+03
Data variables:
    so          (experiment, time, z, y, x) float32 622MB dask.array<chunksize=(1, 12, 50, 180, 360), meta=np.ndarray>

In [46]:
gridname = str(dlon)+'x'+str(dlat)
savename = '.'.join([config,variable,averaging,gridname,'nc'])
with ProgressBar():
    ds = ds.compute()
ds.to_netcdf('../data/processed/regridded/'+savename,mode='w')

In [48]:
dsnow = xr.open_dataset('../data/processed/regridded/'+savename)
dsnow

<xarray.Dataset> Size: 1GB
Dimensions:     (experiment: 4, time: 12, z: 50, y: 180, x: 360)
Coordinates:
  * experiment  (experiment) <U19 304B 'deepmip_sens_1xCO2' ... 'piControl'
  * time        (time) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * x           (x) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * y           (y) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * z           (z) float64 400B 5.0 15.0 25.0 ... 4.588e+03 4.95e+03 5.316e+03
Data variables:
    so          (experiment, time, z, y, x) float64 1GB ...

In [50]:
dsnow['deepmip_sens_2xCO2'].isel(time=5,z=13).plot()

KeyError: "No variable named 'deepmip_sens_2xCO2'. Variables on the dataset include ['so', 'experiment', 'time', 'x', 'y', 'z']"